# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [3]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
s_data = np.array(student_data)
#print np.sum(s_data[:,30] == 'yes')

#print np.sum(s_data=='Yes')

n_students = s_data.shape[0]
n_features = s_data.shape[1]-1
n_passed = np.sum(s_data[:,30] == 'yes')
n_failed = np.sum(s_data[:,30] == 'no')
grad_rate = (float(n_passed)/float(n_students))*100





print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [4]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows
print y_all.head()

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [5]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)


print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [6]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

x_data_mod = np.array(X_all)
y_data_mod = np.array(y_all)
#print x_data_mod.shape
#print x_data_mod[0]
#print y_data_mod[0]

from sklearn.utils import shuffle

x_data_rand, y_data_rand = shuffle(x_data_mod, y_data_mod, random_state=0)
#print x_data_rand[0]
#print y_data_rand[0]


#np.random.shuffle(s_data_mod)



# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset


X_train = x_data_rand[0:num_train]
X_train_100 = x_data_rand[0:100]
X_train_200 = x_data_rand[0:200]
#print X_train
#print X_train.shape
y_train = y_data_rand[0:num_train]
y_train_100 = y_data_rand[0:100]
y_train_200 = y_data_rand[0:200]
#print y_train
#print y_train.shape
X_test = x_data_rand[num_train:]
y_test = y_data_rand[num_train:]

import matplotlib.pyplot as plt

y_array = []
for item in y_train:
    if item == 'yes':
        y_array.append(1)
    else:
        y_array.append(0)
y_train_mod = np.array(y_array)

y_test_array = []
for item in y_test:
    if item == 'yes':
        y_test_array.append(1)
    else:
        y_test_array.append(0)
y_test_mod = np.array(y_test_array)

#print y_train_mod.shape
#print X_train[:][0].shape
#print X_train[0:3][:]
#print X_train[:][0:5]
#print X_train[:][2]
#print X_train[2][:]
#print X_train[1][4]
#print X_train[4][1]
#print y_train_mod
#print X_train[:].shape
#print X_train[:,47]
#plt.scatter(X_train[:,47], y_train_mod)
#plt.show()

print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [7]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.6f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn import svm
clf = svm.SVC()
#clf = svm.LinearSVC()
# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

Training SVC...
Done!
Training time (secs): 0.006969


In [8]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.6f}".format(end - start)
    return f1_score(target, y_pred, pos_label='yes')

def predict_labels_mod(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.6f}".format(end - start)
    return f1_score(target, y_pred, pos_label=1)

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using SVC...
Done!
Prediction time (secs): 0.004366
F1 score for training set: 0.859688195991


In [9]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using SVC...
Done!
Prediction time (secs): 0.002639
F1 score for test set: 0.853503184713


In [10]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))
    
def train_predict_mod(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels_mod(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels_mod(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data

train_predict(clf, X_train, y_train, X_test, y_test)
train_predict(clf, X_train_100, y_train_100, X_test, y_test)
train_predict(clf, X_train_200, y_train_200, X_test, y_test)
# Note: Keep the test set constant

------------------------------------------
Training set size: 300
Training SVC...
Done!
Training time (secs): 0.008139
Predicting labels using SVC...
Done!
Prediction time (secs): 0.006604
F1 score for training set: 0.859688195991
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002254
F1 score for test set: 0.853503184713
------------------------------------------
Training set size: 100
Training SVC...
Done!
Training time (secs): 0.000989
Predicting labels using SVC...
Done!
Prediction time (secs): 0.000633
F1 score for training set: 0.857142857143
Predicting labels using SVC...
Done!
Prediction time (secs): 0.000588
F1 score for test set: 0.857142857143
------------------------------------------
Training set size: 200
Training SVC...
Done!
Training time (secs): 0.003290
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002179
F1 score for training set: 0.851211072664
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001182
F1 score for test set

In [11]:
# TODO: Train and predict using two other models
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier(random_state=0)
train_predict(dt_clf, X_train, y_train, X_test, y_test)
train_predict(dt_clf, X_train_200, y_train_200, X_test, y_test)
train_predict(dt_clf, X_train_100, y_train_100, X_test, y_test)




------------------------------------------
Training set size: 300
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.002791
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000305
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000303
F1 score for test set: 0.776119402985
------------------------------------------
Training set size: 200
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001618
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000207
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000149
F1 score for test set: 0.766917293233
------------------------------------------
Training set size: 100
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001054
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000183
F1 

In [12]:
from sklearn.naive_bayes import GaussianNB

nb_clf = GaussianNB()
train_predict(nb_clf, X_train, y_train, X_test, y_test)
train_predict(nb_clf, X_train_200, y_train_200, X_test, y_test)
train_predict(nb_clf, X_train_100, y_train_100, X_test, y_test)

------------------------------------------
Training set size: 300
Training GaussianNB...
Done!
Training time (secs): 0.001916
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000560
F1 score for training set: 0.810810810811
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000272
F1 score for test set: 0.712121212121
------------------------------------------
Training set size: 200
Training GaussianNB...
Done!
Training time (secs): 0.001319
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000413
F1 score for training set: 0.7969348659
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000296
F1 score for test set: 0.738461538462
------------------------------------------
Training set size: 100
Training GaussianNB...
Done!
Training time (secs): 0.001240
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000669
F1 score for training set: 0.814285714286
Predicting labels using GaussianNB...

In [13]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier()
train_predict(sgd_clf, X_train, y_train, X_test, y_test)
train_predict(sgd_clf, X_train_100, y_train_100, X_test, y_test)
train_predict(sgd_clf, X_train_200, y_train_200, X_test, y_test)

------------------------------------------
Training set size: 300
Training SGDClassifier...
Done!
Training time (secs): 0.000847
Predicting labels using SGDClassifier...
Done!
Prediction time (secs): 0.000261
F1 score for training set: 0.790224032587
Predicting labels using SGDClassifier...
Done!
Prediction time (secs): 0.000122
F1 score for test set: 0.858895705521
------------------------------------------
Training set size: 100
Training SGDClassifier...
Done!
Training time (secs): 0.001399
Predicting labels using SGDClassifier...
Done!
Prediction time (secs): 0.000618
F1 score for training set: 0.802547770701
Predicting labels using SGDClassifier...
Done!
Prediction time (secs): 0.000122
F1 score for test set: 0.85534591195
------------------------------------------
Training set size: 200
Training SGDClassifier...
Done!
Training time (secs): 0.000500
Predicting labels using SGDClassifier...
Done!
Prediction time (secs): 0.000067
F1 score for training set: 0.774907749077
Predicting l

In [14]:
from sklearn.linear_model import LogisticRegression
lreg_clf = LogisticRegression()
train_predict(lreg_clf, X_train, y_train, X_test, y_test)

------------------------------------------
Training set size: 300
Training LogisticRegression...
Done!
Training time (secs): 0.002891
Predicting labels using LogisticRegression...
Done!
Prediction time (secs): 0.000195
F1 score for training set: 0.816425120773
Predicting labels using LogisticRegression...
Done!
Prediction time (secs): 0.000383
F1 score for test set: 0.788732394366


In [15]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
train_predict(neigh, X_train, y_train, X_test, y_test)


------------------------------------------
Training set size: 300
Training KNeighborsClassifier...
Done!
Training time (secs): 0.001531
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.007478
F1 score for training set: 0.859857482185
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.003529
F1 score for test set: 0.783216783217


## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [16]:
# TODO: Fine-tune your model and report the best F1 score

In [21]:
from sklearn import grid_search
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer

def createCount(maxCount):
    newCount = []
    for i in range(maxCount):
        a = i+1
        newCount.append(a)
    return newCount
    
maxDepth = createCount(7)
minSplit = createCount(19)
minLeaf = createCount(1)
myCount=1
dtc0 =DecisionTreeClassifier(criterion='gini', min_samples_split=1, min_samples_leaf=1)
dtc = DecisionTreeClassifier(criterion='gini', random_state=0, max_depth=5, min_samples_split=20, min_samples_leaf=1)
dtc2 = DecisionTreeClassifier(max_depth=4, min_samples_split=1, min_samples_leaf=1)
treeScorer = make_scorer(f1_score, pos_label="yes", greater_is_better=True)
parameters = {'max_depth': myCount,
             'min_samples_split': myCount,
             'min_samples_leaf': myCount}
simple_param = {'max_depth': maxDepth,
               'min_samples_split': minSplit,
               'min_samples_leaf': minLeaf}
simple_param2 = {'max_depth': myCount}
dtc_tuned = grid_search.GridSearchCV(dtc0, cv=5, param_grid=simple_param, scoring=treeScorer)
"""
dtc_test = grid_search.GridSearchCV(DecisionTreeClassifier(),
                                   param_grid={"max_depth": [1,5],
                              "min_samples_leaf": [1]}, scoring=treeScorer)
                              """
#train_classifier(dtc_tuned, X_train, y_train_mod)
train_predict(dtc, X_train, y_train, X_test, y_test)
train_predict(dtc2, X_train, y_train, X_test, y_test)
train_predict(dtc_tuned, X_train, y_train, X_test, y_test)
#train_predict(dtc, X_train_200, y_train_200, X_test, y_test)
#train_predict(dtc, X_train_100, y_train_100, X_test, y_test)





------------------------------------------
Training set size: 300
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001428
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000102
F1 score for training set: 0.859090909091
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000140
F1 score for test set: 0.851351351351
------------------------------------------
Training set size: 300
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.000927
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000096
F1 score for training set: 0.87619047619
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000062
F1 score for test set: 0.829931972789
------------------------------------------
Training set size: 300
Training GridSearchCV...
Done!
Training time (secs): 1.715187
Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 0.000128
F1